<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#透视表" data-toc-modified-id="透视表-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>透视表</a></span></li><li><span><a href="#交叉表，是数据透视表的一个特殊情况，计算的是分组中的频率" data-toc-modified-id="交叉表，是数据透视表的一个特殊情况，计算的是分组中的频率-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>交叉表，是数据透视表的一个特殊情况，计算的是分组中的频率</a></span></li></ul></div>

pivot_table():

参数 | 说明
---|---
values | 聚合列，默认情况下，排除冗余列（非数值列），聚合所有数值型的列
index | 在透视表的行上进行分组的列名
columns | 在透视表上的列上进行分组的列名
aggfunc | 聚合函数（默认为'mean'）
fill_value | 在透视表中替换缺失值的值
dropna | 如果为True，删除所有条目均为NA的列
margins | 总计

## 透视表

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
# 读取数据集
tips = pd.read_csv('examples/tips.csv')

In [3]:
# 添加一列，表示小费的比例
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [4]:
# 前6行
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [5]:
# 将day列和smoker列转换为index
# time列为冗余列，在结果中不显示
tips.pivot_table(index=['day', 'smoker']) # 默认聚合函数为mean

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [6]:
tips.groupby(['day',  'smoker'])[['size', 'tip', 'tip_pct', 'total_bill']].mean()

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [7]:
# 将day列和smoker列转换为index
tips.pivot_table(index=['day', 'smoker'],
                 aggfunc='mean') # 默认聚合函数为mean

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [8]:
tips.pivot_table(['tip_pct', 'size'], # 聚合列
                 index=['time', 'day'], # 行索引
                 columns='smoker') # 列索引

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [9]:
tips.groupby(['time', 'day', 'smoker'])[['tip_pct', 'size']].mean().unstack('smoker')

tip_pct                size          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   0.139622  0.165347  2.000000  2.222222
       Sat   0.158048  0.147906  2.555556  2.476190
       Sun   0.160113  0.187250  2.929825  2.578947
       Thur  0.159744       NaN  2.000000       NaN
Lunch  Fri   0.187735  0.188937  3.000000  1.833333
       Thur  0.160311  0.163863  2.500000  2.352941

In [10]:
tips.pivot_table(['tip_pct', 'size'], # 聚合列
                 index=['time', 'day'], # 行索引
                 columns='smoker', # 列索引
                 margins=True) # 总计项

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [11]:
tips.groupby(['time', 'day', 'smoker'])[['tip_pct', 'size']].mean().unstack('smoker')

tip_pct                size          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   0.139622  0.165347  2.000000  2.222222
       Sat   0.158048  0.147906  2.555556  2.476190
       Sun   0.160113  0.187250  2.929825  2.578947
       Thur  0.159744       NaN  2.000000       NaN
Lunch  Fri   0.187735  0.188937  3.000000  1.833333
       Thur  0.160311  0.163863  2.500000  2.352941

In [12]:
tips.pivot_table('tip_pct', # 聚合列
                 index=['time', 'smoker'], # 行索引
                 columns='day', # 列索引
                 aggfunc=len, # 聚合函数为len，表示计数
                 margins=True) # 总计

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [13]:
tips.pivot_table('tip_pct', # 聚合列
                 index=['time', 'smoker'], # 行索引
                 columns='day', # 列索引
                 aggfunc='count', # 聚合函数为'count'，表示计数
                 margins=True) # 总计

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [14]:
tips.groupby(['time', 'smoker', 'day'])['tip_pct'].count().unstack('day')

day            Fri   Sat   Sun  Thur
time   smoker                       
Dinner No      3.0  45.0  57.0   1.0
       Yes     9.0  42.0  19.0   NaN
Lunch  No      1.0   NaN   NaN  44.0
       Yes     6.0   NaN   NaN  17.0

In [15]:
tips.pivot_table('tip_pct', # 聚合列
                 index=['time', 'size', 'smoker'], # 行索引
                 columns='day', # 列索引
                 aggfunc='mean', # 聚合函数
                 fill_value=0) # 用0来填充缺失值

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

## 交叉表，是数据透视表的一个特殊情况，计算的是分组中的频率

In [16]:
data = DataFrame({'Sample': range(1, 11),
                  'Nationality': ['USA', 'Japan', 'USA', 'Japan', 'Japan', 'Japan', 'USA', 'USA', 'Japan', 'USA'],
                  'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 
                                 'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed']})

In [17]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [18]:
pd.crosstab(index=data.Nationality, # 行
            columns=data.Handedness, # 列
            margins=True) # 小计

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [19]:
data.groupby(['Nationality', 'Handedness']).size().unstack(1)

Handedness,Left-handed,Right-handed
Nationality,,
Japan,2,3
USA,1,4


In [20]:
pd.crosstab(index=[tips.time, tips.day], # 行
            columns=tips.smoker, # 列
            margins=True) # 小计

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [21]:
tips.groupby(['time', 'day', 'smoker']).size().unstack('smoker')

smoker         No   Yes
time   day             
Dinner Fri    3.0   9.0
       Sat   45.0  42.0
       Sun   57.0  19.0
       Thur   1.0   NaN
Lunch  Fri    1.0   6.0
       Thur  44.0  17.0